### 🛠️ Initialize Notebook Variables

**Only modify entries under _USER CONFIGURATION_.**

In [ ]:
import utils
from apimtypes import *
from console import print_error, print_info, print_ok, print_val, print_warning
from azure_resources import get_infra_rg_name, get_account_info

# ------------------------------
#    USER CONFIGURATION
# ------------------------------

rg_location = 'eastus2'
index       = 2
apim_sku    = APIM_SKU.BASICV2              # Options: 'BASICV2', 'STANDARDV2', 'PREMIUMV2'
deployment  = INFRASTRUCTURE.SIMPLE_APIM    # Options: see supported_infras below
api_prefix  = 'costing-'                    # ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES
tags        = ['costing', 'cost-management', 'observability']  # ENTER DESCRIPTIVE TAG(S)

# Cost export configuration
cost_export_frequency = 'Daily'             # Options: 'Daily', 'Weekly', 'Monthly'

# Sample data generation
generate_sample_load = True                 # Generate sample API calls to demonstrate cost tracking
sample_requests_per_subscription = 50       # Base request count per business unit (multiplied by each BU's weight)

# Budget alerts
alert_threshold = 1000                      # Request count threshold per BU per hour
alert_email = 'alerts@contoso.com'          # Email for alert notifications (leave empty to skip)



# ------------------------------
#    SYSTEM CONFIGURATION
# ------------------------------

sample_folder    = 'costing'
rg_name          = get_infra_rg_name(deployment, index)
supported_infras = [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.APPGW_APIM, INFRASTRUCTURE.APPGW_APIM_PE, INFRASTRUCTURE.SIMPLE_APIM]
nb_helper        = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, supported_infras, True, index = index, apim_sku = apim_sku)

# Define the API and its operations
api_path = 'cost-demo'
cost_demo_get = GET_APIOperation2('get-status', 'Get Status', '/get', 'Get Status')

apis = [
    API(f'{api_prefix}cost-tracking-api', 'Cost Tracking Demo API', api_path,
        'API for demonstrating cost tracking and allocation',
        operations = [cost_demo_get], tags = tags, subscriptionRequired = True, serviceUrl = 'https://httpbin.org')
]

# Define business units
business_units = [
    {'name': 'bu-hr', 'display': 'Business Unit - Human Resources', 'request_weight': 1.0},
    {'name': 'bu-finance', 'display': 'Business Unit - Finance', 'request_weight': 2.5},
    {'name': 'bu-marketing', 'display': 'Business Unit - Marketing', 'request_weight': 0.5},
    {'name': 'bu-engineering', 'display': 'Business Unit - Engineering', 'request_weight': 3.0}
]

# Get Azure account information
current_user, current_user_id, tenant_id, subscription_id = get_account_info()

if not subscription_id:
    print_error('Could not determine Azure subscription ID. Run: az login')
    raise SystemExit(1)

print_ok('Notebook initialized')

### 🚀 Deploy Infrastructure and APIs

Creates the bicep deployment into the previously-specified resource group. A bicep parameters, `params.json`, file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'location'              : {'value': rg_location},
    'costExportFrequency'   : {'value': cost_export_frequency},
    'index'                 : {'value': index},
    'apis'                  : {'value': [api.to_dict() for api in apis]},
    'businessUnits'         : {'value': [{'name': bu['name'], 'displayName': bu['display']} for bu in business_units]}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs
    apim_name                      = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url               = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    app_insights_name              = output.get('applicationInsightsName', 'Application Insights Name')
    app_insights_connection_string = output.get('applicationInsightsConnectionString', '')
    log_analytics_name             = output.get('logAnalyticsWorkspaceName', 'Log Analytics Workspace Name')
    storage_account_name           = output.get('storageAccountName', 'Storage Account Name')
    workbook_name                  = output.get('workbookName', 'Workbook Name')
    workbook_id                    = output.get('workbookId', '')
    cost_export_name               = f'apim-cost-export-{index}-{rg_name}'

    # Extract subscription keys
    subscription_keys_output = output.getJson('subscriptionKeys', 'Subscription Keys', secure=True)

    # Map keys to business units
    subscriptions = {}
    if subscription_keys_output:
        for bu in business_units:
            sub_id = bu['name']
            primary_key = next((item['primaryKey'] for item in subscription_keys_output if item['name'] == sub_id), None)

            subscriptions[sub_id] = {
                'display_name': bu['display'],
                'primary_key': primary_key,
                'request_weight': bu.get('request_weight', 1.0)
            }

    print_ok('Deployment completed successfully')
else:
    print_error("Deployment failed!")
    raise SystemExit(1)

### 🔧 Configure Cost Management Export

Automatically set up cost data export to the storage account.

In [ ]:
import json
import tempfile
from datetime import datetime, timedelta, timezone
from pathlib import Path

from azure_resources import run

if 'storage_account_name' not in locals():
    print_error('Please run the deployment cell first')
    raise SystemExit(1)

print_info('Configuring automated Cost Management export (managed identity)...')

# Get storage account resource ID
storage_account_id = (
    f'/subscriptions/{subscription_id}'
    f'/resourceGroups/{rg_name}'
    f'/providers/Microsoft.Storage/storageAccounts/{storage_account_name}'
)

# Export scope and name
export_scope = f'/subscriptions/{subscription_id}'
api_version = '2025-03-01'

# Register required resource provider
print_info('Registering Microsoft.CostManagementExports resource provider...')
register_result = run(
    'az provider register --namespace Microsoft.CostManagementExports --wait',
    log_command=False
)

if register_result.success:
    print_ok('Resource provider registered successfully')

# Check if export already exists
existing_export = run(
    f'az rest --method GET '
    f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{cost_export_name}'
    f'?api-version={api_version}" -o json',
    log_command=False
)

if existing_export.success:
    print_warning(f'Cost export "{cost_export_name}" already exists - recreating...')
    run(
        f'az rest --method DELETE '
        f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{cost_export_name}'
        f'?api-version={api_version}"',
        log_command=False
    )

# Build recurrence settings
recurrence_map = {'Daily': 'Daily', 'Weekly': 'Weekly', 'Monthly': 'Monthly'}
recurrence = recurrence_map.get(cost_export_frequency, 'Daily')

start_date = (datetime.now(timezone.utc) + timedelta(days=1)).strftime('%Y-%m-%dT00:00:00Z')
end_date = (datetime.now(timezone.utc) + timedelta(days=365)).strftime('%Y-%m-%dT00:00:00Z')

# Build the export body with system-assigned managed identity
export_body = {
    'identity': {
        'type': 'systemAssigned'
    },
    'location': 'global',
    'properties': {
        'definition': {
            'type': 'ActualCost',
            'timeframe': 'MonthToDate',
            'dataSet': {
                'granularity': 'Daily'
            }
        },
        'deliveryInfo': {
            'destination': {
                'type': 'AzureBlob',
                'container': 'cost-exports',
                'rootFolderPath': 'apim-costing',
                'resourceId': storage_account_id
            }
        },
        'schedule': {
            'status': 'Active',
            'recurrence': recurrence,
            'recurrencePeriod': {
                'from': start_date,
                'to': end_date
            }
        },
        'format': 'Csv'
    }
}

print_info('Creating cost export with managed identity...')

# Write body to a temp file for cross-platform compatibility
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as body_file:
    json.dump(export_body, body_file)
    body_file_path = body_file.name

try:
    export_result = run(
        f'az rest --method PUT '
        f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{cost_export_name}'
        f'?api-version={api_version}" '
        f'--body @{body_file_path} -o json',
        log_command=False
    )
finally:
    Path(body_file_path).unlink(missing_ok=True)

if export_result and export_result.success:
    print_ok(f'Cost export created: {cost_export_name}')
    print_val('Export frequency', recurrence)
    print_val('Authentication', 'System-assigned managed identity')
    cost_export_configured = True

    # Extract the managed identity principal ID from the response
    export_data = json.loads(export_result.text)
    principal_id = export_data.get('identity', {}).get('principalId')

    if principal_id:
        print_info('Assigning Storage Blob Data Contributor role to export identity...')

        role_assignment = run(
            f'az role assignment create '
            f'--assignee-object-id {principal_id} '
            f'--assignee-principal-type ServicePrincipal '
            f'--role "Storage Blob Data Contributor" '
            f'--scope {storage_account_id}',
            log_command=False
        )

        if role_assignment.success:
            print_ok('Storage Blob Data Contributor role assigned to export identity')
        else:
            print_warning('Could not assign role - you may need to do this manually')
    else:
        print_warning('Could not retrieve export identity principal ID')

    print_info('Cost data will be exported automatically starting tomorrow')
else:
    print_error('Failed to create cost export')
    print_warning('Continuing without cost export - you can configure it manually later')
    cost_export_configured = False

### 📤 Trigger Initial Cost Export

Run the first cost export manually to populate data immediately.

In [ ]:
from azure_resources import run

if 'cost_export_configured' not in locals():
    print_error('Please run the cost export configuration cell first')
    raise SystemExit(1)

if cost_export_configured:
    export_scope = f'/subscriptions/{subscription_id}'
    api_version = '2025-03-01'

    print_info(f'Triggering first cost export run for "{cost_export_name}"...')

    run_result = run(
        f'az rest --method POST '
        f'--url "{export_scope}/providers/Microsoft.CostManagement/exports/{cost_export_name}'
        f'/run?api-version={api_version}"',
        log_command=False
    )

    if run_result.success:
        print_ok('Cost export run triggered successfully')
        print_info('Data will appear in the storage container within a few minutes')
    else:
        print_warning('Could not trigger export run - it will run on its next scheduled recurrence')
else:
    print_warning('Cost export was not configured - skipping manual run')

### 🚀 Generate Sample API Traffic

Generate sample API calls from each business unit subscription to demonstrate cost tracking and allocation.

This will create request logs in Application Insights and Log Analytics that can be used for cost analysis.

In [ ]:
from apimrequests import ApimRequests

if 'apim_gateway_url' not in locals():
    print_error('Please run the deployment cell first')
    raise SystemExit(1)

if generate_sample_load:
    print_info('Generating sample API traffic...')

    # Determine endpoints, URLs, etc. prior to test execution
    endpoint_url, request_headers = utils.get_endpoint(deployment, rg_name, apim_gateway_url)

    # Send requests for each business unit, weighted by its configured request_weight
    for subscription_id_sub, sub_info in subscriptions.items():
        bu_request_count = max(1, int(sample_requests_per_subscription * sub_info.get('request_weight', 1.0)))

        reqs = ApimRequests(endpoint_url, sub_info['primary_key'], request_headers)
        reqs.multiGet(f'/{api_path}/get', bu_request_count, msg = f'Generating {bu_request_count} requests for {subscription_id_sub}', printResponse = False, sleepMs = 10)

    print_info('Note: It may take 2-5 minutes for logs to appear in Application Insights and Log Analytics')
else:
    print_info('Sample load generation skipped (generate_sample_load = False)')

### 🔍 Verify Log Ingestion

Waits for diagnostic logs to arrive in Log Analytics (auto-retries for up to 10 minutes).

In [ ]:
import json
import tempfile
import time
from pathlib import Path

from azure_resources import run

if 'log_analytics_name' not in locals():
    print_error('Please run the deployment cell first')
    raise SystemExit(1)

print_info('Waiting for APIM logs to arrive in Log Analytics...')
print_info('Log ingestion typically takes 2-5 minutes after generating traffic')
print()

print_val('Workspace', log_analytics_name)

# Build the workspace resource ID for the ARM query endpoint
workspace_resource_id = (
    f'/subscriptions/{subscription_id}'
    f'/resourceGroups/{rg_name}'
    f'/providers/Microsoft.OperationalInsights/workspaces/{log_analytics_name}'
)

# Load KQL from external file and wrap it in a JSON body
kql_path = utils.determine_policy_path('verify-log-ingestion.kql', sample_folder)
kql_query = Path(kql_path).read_text(encoding='utf-8')

query_body = {'query': kql_query}

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump(query_body, f)
    query_file_path = f.name

# Poll Log Analytics until gateway logs with subscription IDs appear
max_wait_minutes = 10
poll_interval_seconds = 30
max_attempts = (max_wait_minutes * 60) // poll_interval_seconds
logs_found = False

try:
    for attempt in range(1, max_attempts + 1):
        result = run(
            f'az rest --method POST '
            f'--url "https://management.azure.com{workspace_resource_id}/api/query?api-version=2020-08-01" '
            f'--body @{query_file_path} -o json',
            log_command=False
        )

        # A non-transient error (e.g. bad API version, auth failure) should stop immediately
        if not result.success:
            print_error(f'Query failed: {result.text[:300]}')
            break

        # Parse the tabular response and check whether any rows were returned
        if result.json_data:
            tables = result.json_data.get('tables', [])
            if tables:
                rows = tables[0].get('rows', [])
                if rows and len(rows) > 0:
                    row_count = int(rows[0][0])
                    if row_count > 0:
                        print_ok(f'Found {row_count} log entries with subscription IDs')
                        logs_found = True
                        break

        elapsed = attempt * poll_interval_seconds
        remaining = (max_wait_minutes * 60) - elapsed
        print_info(f'  No logs yet... retrying in {poll_interval_seconds}s ({remaining}s remaining)')
        time.sleep(poll_interval_seconds)
finally:
    Path(query_file_path).unlink(missing_ok=True)

if logs_found:
    print_ok('Log ingestion verified - workbook should now display data')
elif result.success:
    print_warning(f'Logs did not appear within {max_wait_minutes} minutes')
    print_info('This can happen with newly created workspaces. Tips:')
    print_info('  1. Wait a few more minutes and re-run this cell')
    print_info('  2. Verify diagnostic settings in Azure Portal')
    print_info('  3. Re-run the traffic generation cell to send more requests')

### 📊 Cost Analysis & Sample Kusto Queries

#### Cost Allocation Model

| Component | Formula |
|---|---|
| **Base Cost** | Monthly platform cost for the APIM SKU |
| **Base Cost Share** | `Base Monthly Cost × (BU Requests ÷ Total Requests)` |
| **Variable Cost** | `BU Requests × (Rate per 1K ÷ 1000)` |
| **Total Allocated** | `Base Cost Share + Variable Cost` |

The next cell uses reasonable defaults. For current pricing, see the [Azure Retail Prices API](https://learn.microsoft.com/rest/api/cost-management/retail-prices/azure-retail-prices).

In [ ]:
# Store in variables for use by other cells
base_monthly_cost   = 150.00
per_k_rate          = 0.003
included_requests_k = 10000
print_val('Base monthly cost (default)', f'${base_monthly_cost:.2f}')
print_val('Overage rate per 1K (default)', f'${per_k_rate}')

# Sample Kusto queries for cost analysis
print()
print_info('Sample Kusto Queries for Log Analytics:')
print_info('These queries use the ApiManagementGatewayLogs table (resource-specific mode).')
print()

### 🔔 Set Up Budget Alerts per Business Unit

Create Azure Monitor scheduled query alerts that fire when a business unit subscription exceeds a configurable request threshold.

Each alert:
- Runs a Kusto query every **5 minutes** against the Log Analytics workspace
- Triggers when a business unit exceeds the threshold in a **1-hour** rolling window
- Sends notifications via an **Action Group** (email)

> Adjust `alert_threshold` and `alert_email` in the initialization cell to match your requirements.

In [ ]:
import json
import tempfile
from pathlib import Path

from azure_resources import run

if not alert_email:
    print_warning('No alert_email configured - skipping budget alert setup')
    print_info('Set alert_email above to enable budget alerts per business unit')
else:
    print_info('Setting up budget alerts per business unit subscription...')

    # Get Log Analytics workspace resource ID
    workspace_result = run(
        f'az monitor log-analytics workspace show '
        f'--resource-group {rg_name} '
        f'--workspace-name {log_analytics_name} '
        f'--query id -o tsv',
        log_command=False
    )
    workspace_id = workspace_result.text.strip()

    # Create an Action Group for alert notifications
    action_group_name = f'ag-apim-cost-alerts-{index}'
    print_info(f'Creating action group: {action_group_name}...')

    ag_result = run(
        f'az monitor action-group create '
        f'--resource-group {rg_name} '
        f'--name {action_group_name} '
        f'--short-name apimcost '
        f'--action email cost-alert-email {alert_email} '
        f'-o json',
        log_command=False
    )

    if ag_result.success:
        action_group_id = ag_result.json_data.get('id', '')
        print_ok(f'Action group created: {action_group_name}')
    else:
        print_error(f'Failed to create action group: {ag_result.text}')
        action_group_id = None

    if action_group_id:
        # Load the KQL template from an external file
        kql_path = utils.determine_policy_path('budget-alert-threshold.kql', sample_folder)
        kql_template = Path(kql_path).read_text(encoding='utf-8')

        bu_list = list(subscriptions.keys())
        if not bu_list:
            bu_list = ['bu-hr', 'bu-finance', 'bu-marketing', 'bu-engineering']

        print_info(f'Creating alerts for {len(bu_list)} business units (threshold: {alert_threshold} requests/hour)...')

        for bu_name in bu_list:
            alert_name = f'apim-budget-{bu_name}-{index}'

            # Prepend KQL let bindings to parameterise the query
            kusto_query = f"let buName = '{bu_name}';\nlet threshold = {alert_threshold};\n{kql_template}"

            alert_body = {
                'location': rg_location,
                'properties': {
                    'displayName': f'APIM Budget Alert: {bu_name}',
                    'description': f'Fires when {bu_name} exceeds {alert_threshold} API requests per hour',
                    'severity': 2,
                    'enabled': True,
                    'evaluationFrequency': 'PT5M',
                    'windowSize': 'PT1H',
                    'scopes': [workspace_id],
                    'criteria': {
                        'allOf': [
                            {
                                'query': kusto_query,
                                'timeAggregation': 'Count',
                                'operator': 'GreaterThan',
                                'threshold': 0,
                                'failingPeriods': {
                                    'numberOfEvaluationPeriods': 1,
                                    'minFailingPeriodsToAlert': 1
                                }
                            }
                        ]
                    },
                    'actions': {
                        'actionGroups': [action_group_id]
                    }
                }
            }

            alert_id = (
                f'/subscriptions/{subscription_id}'
                f'/resourceGroups/{rg_name}'
                f'/providers/Microsoft.Insights/scheduledQueryRules/{alert_name}'
            )

            # Write body to a temp file to avoid shell quoting issues on Windows
            with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
                json.dump(alert_body, f)
                alert_body_path = f.name

            try:
                result = run(
                    f'az rest --method PUT '
                    f'--uri https://management.azure.com{alert_id}?api-version=2023-03-15-preview '
                    f'--body @{alert_body_path}',
                    log_command=False
                )
            finally:
                Path(alert_body_path).unlink(missing_ok=True)

            if result.success:
                print_ok(f'  Alert created: {alert_name}')
            else:
                print_error(f'  Failed to create alert for {bu_name}: {result.text[:200]}')

        print()
        print_ok('Budget alerts configured!')
        print_val('Action Group', action_group_name)
        print_val('Alert Email', alert_email)
        print_val('Threshold', f'{alert_threshold} requests per hour per BU')
        print_val('Evaluation', 'Every 5 minutes, 1-hour rolling window')

### 🔗 Access Your Resources

Links to access the deployed resources in Azure Portal.

In [ ]:
print_info('📎 Azure Portal Links:')
print()

base_url = 'https://portal.azure.com/#@/resource'
subscription_path = f'/subscriptions/{subscription_id}'
rg_path = f'{subscription_path}/resourceGroups/{rg_name}'

resources = [
    ('Application Insights', f'{rg_path}/providers/Microsoft.Insights/components/{app_insights_name}/overview'),
    ('Log Analytics Workspace', f'{rg_path}/providers/Microsoft.OperationalInsights/workspaces/{log_analytics_name}/Overview'),
    ('Storage Account', f'{rg_path}/providers/Microsoft.Storage/storageAccounts/{storage_account_name}/overview'),
    ('APIM Service', f'{rg_path}/providers/Microsoft.ApiManagement/service/{apim_name}/overview'),
    ('Cost Management + Exports', f'{subscription_path}/costManagement')
]

if 'workbook_id' in locals():
    resources.append(('Azure Monitor Workbook', f'{base_url}{workbook_id}/workbook'))

for name, path in resources:
    print(f'{name}:')
    print(f'{base_url}{path}')
    print()

print_ok('Setup complete!')
print()
print_info('📊 Cost Allocation is now automated:')
print('  ✓ Request logs flowing to Application Insights & Log Analytics')
if cost_export_configured:
    print('  ✓ Cost data automatically exported daily to Storage Account')
else:
    print('  ⚠ Cost export not configured - configure manually via Azure Portal')
print('  ✓ Azure Monitor Workbook ready for cost visualization')
print('  ✓ Business unit usage tracked via APIM subscriptions')
print()
print_info('💡 Cost allocation workflow:')
print('  1. APIM logs each API request with subscription info')
print('  2. Azure exports daily cost data to storage')
print('  3. Correlate usage metrics with costs using subscription IDs')
print('  4. Create chargeback reports per business unit')
print('  5. Set up budget alerts based on usage patterns')
print()
print_info('🔍 Next steps:')
print('  • Open Azure Monitor Workbook to view real-time cost dashboard')
print('  • Query Log Analytics for detailed usage analysis')
print('  • Review Cost Management exports (available after first export run)')
print()
print_info('To clean up resources, open and run: clean-up.ipynb')